In [ ]:
%pip install flask-ngrok
%pip install pytesseract

!apt install tesseract-ocr
!apt install libtesseract-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (5,171 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 121753 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
!pip install pytesseract

In [ ]:
!pip install tesseract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 MB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tesseract: filename=tesseract-0.1.3-py3-none-any.whl size=45562552 sha256=a1e734e54b3b0f4738fc4788adc094c6983a449281a400ebdb51894411b982f0
  Stored in directory: /root/.cache/pip/wheels/71/c9/aa/698c579693e83fdda9ad6d6f0d8f61ed986e27925ef576f109
Successfully built tesseract


In [ ]:
import cv2
import matplotlib.pyplot as plt
from collections import Counter
from PIL import Image, ImageDraw
from PIL import ImagePath
import numpy as np
import os
import pandas as pd
import json
import tensorflow as tf
import pytesseract
import shutil
# from google.colab.patches import cv2_imshow

In [ ]:
!gdown --id "1525_Uvl9VB61O4Jt70RYGRMDOk9qXCnu"
!unzip -qq tablenet_model.zip


/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1525_Uvl9VB61O4Jt70RYGRMDOk9qXCnu
From (redirected): https://drive.google.com/uc?id=1525_Uvl9VB61O4Jt70RYGRMDOk9qXCnu&confirm=t&uuid=768b31fe-a6cd-41f0-9adc-aa1b66a186d1
To: /content/tablenet_model.zip
100% 75.1M/75.1M [00:00<00:00, 117MB/s]


In [ ]:
!gdown --id "1ij4_eVrAIGRZBhVwtBNBgMZ7bADOWNu7"
!unzip -qq new.zip


/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1ij4_eVrAIGRZBhVwtBNBgMZ7bADOWNu7
To: /content/new.rar
100% 75.4M/75.4M [00:00<00:00, 192MB/s]
unzip:  cannot find or open new.zip, new.zip.zip or new.zip.ZIP.


In [ ]:
# from google.colab import drive
# import os

# # Mount Google Drive
# drive.mount('/content/drive')



In [ ]:
# import os

# # Navigate to the directory containing the zip file
# os.chdir('/content/drive/MyDrive/')

# # Unzip the file
# !unzip -q /content/drive/MyDrive/tablenet_model.zip -d /content/


In [ ]:
model = tf.keras.models.load_model('./saved_model')

In [ ]:
import os
import shutil

# Function to delete folder contents if it exists
def delete_folder_contents(folder_path):
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)

# Paths for folders
temp_folder = "/content/temp"
output_folder = "/content/output"

# Delete contents of folders if they exist
delete_folder_contents(temp_folder)
delete_folder_contents(output_folder)

# Create folders
os.mkdir(temp_folder)
os.mkdir(output_folder)

# Helper Methods

In [ ]:
# given predicted boxes approximate the predicted rectangles
def fil_approx_boxes(img):
    cv2.imwrite("/content/temp/test.jpeg",img)
    img = cv2.imread("/content/temp/test.jpeg",0)
    img = cv2.medianBlur(img,5)
    img = cv2.GaussianBlur(img,(13,13),0)
    img = cv2.threshold(img, 254, 255, cv2.THRESH_BINARY)[1]

    _, threshold = cv2.threshold(img, 254, 255, cv2.THRESH_BINARY_INV)
    contours,_ = cv2.findContours(threshold, cv2.RETR_TREE , cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        if x==0 or y==0:
            continue
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,255),-1)

    img = cv2.GaussianBlur(img,(13,13),0)
    img = cv2.threshold(img, 254, 255, cv2.THRESH_BINARY)[1]
    return img

In [ ]:
# Save data into respective CSV Files
def save_to_csv(csv_name,data):
  delim = " "
  if data.find(",") != -1:
    delim = ","
  elif data.find("|") != -1:
    delim = "|"

  data_arr = data.split("\n")
  data_arr = [arr for arr in data_arr if len(arr.strip()) != 0]
  with open(csv_name+".csv",'w') as file:
    for line in data_arr:
      line = line.replace(delim,",")
      file.write(line+"\n")
    file.close()


In [ ]:
#  Given masked image, Save both tables and extract text from each
def extract_text(img_path="/content/temp/final_masked.jpeg"):
  img = cv2.imread(img_path,0)
  org_img = img
  img = cv2.GaussianBlur(img,(13,13),0)
  img = cv2.threshold(img, 0,255, cv2.THRESH_BINARY)[1]
  kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])

  _, threshold = cv2.threshold(img, 254, 255, cv2.THRESH_BINARY_INV)
  contours,_ = cv2.findContours(threshold, cv2.RETR_TREE , cv2.CHAIN_APPROX_SIMPLE)
  idx = 1
  for cnt in contours:
    file_name = "/content/output/Table_"+str(idx)
    x,y,w,h = cv2.boundingRect(cnt)
    if x==0 or y==0 or w*h < 20000:
        continue

    roi = org_img[y:y+h, x:x+w]
    roi = cv2.filter2D(roi, -1, kernel)
    roi = cv2.resize(roi, (int(w*1.25),int(h*1.25)), interpolation = cv2.INTER_AREA)
    # cv2_imshow(roi)
    data = pytesseract.image_to_string(roi,config='--psm 6',lang='eng')
    cv2.imwrite(file_name+".jpeg",roi)
    save_to_csv(file_name,data)

    idx += 1


In [ ]:
# predict table and column masks and display
def predict_table_masks(img):
  res1, res2 = model.predict(np.array([img]))
  res1 =  np.expand_dims(np.argmax(res1[0], axis=-1), axis=-1)
  res2 = np.expand_dims(np.argmax(res2[0], axis=-1), axis=-1)
  pred_col = np.squeeze(np.where(res1==1,255,0))
  pred_table = np.squeeze(np.where(res2==1,255,0))

  return fil_approx_boxes(pred_table),fil_approx_boxes(pred_col)


In [ ]:
import time
# Predict masks and extract text
def predict_and_extract(img_path):
  for file_name in os.listdir("/content/output"):
    os.remove("/content/output/"+file_name)

  image = tf.io.read_file(img_path)
  org_image = tf.image.decode_image(image, channels=3)
  h,w = org_image.shape[0],org_image.shape[1]

  image = tf.image.resize(org_image, [800, 800])
  pred_table, pred_col = predict_table_masks(image)
  tab = np.where(pred_table == 0,0,1)
  mask = np.expand_dims(tab,axis=2)
  mask = np.concatenate((mask,mask,mask),axis=2)
  cv2.imwrite("/content/temp/mask.jpeg",mask)

  mask = cv2.resize(cv2.imread("/content/temp/mask.jpeg"), (w,h), interpolation = cv2.INTER_AREA)
  masked_img= org_image.numpy() * mask
  cv2.imwrite("/content/temp/final_masked.jpeg",masked_img)
  extract_text()
  shutil.make_archive('output', 'zip', "output/")
  time.sleep(3)

In [ ]:
# !apt-get install tesseract-ocr


In [ ]:
# predict_and_extract("/content/A.bmp")

# Flask Server

In [ ]:
# !pip install flask
# !pip install pyngrok
# AUTH :  2eKHWPgvraFcBFPr9WWFNaIHBQW_6tMCgAaAQQZ7wjjyEVzZm

In [ ]:
!pip install pyngrok

In [ ]:
import getpass
import os
import time
from flask import Flask, render_template, request, send_from_directory, send_file
from flask import redirect, render_template
from pyngrok import ngrok, conf

print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = getpass.getpass()

app = Flask(__name__)

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(5000).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 5000))

@app.route("/")
def home():
    return render_template("home.html", base_url=request.base_url)

@app.route("/upload", methods=['POST'])
def upload():
    f = request.files['file']
    print(f.filename)
    file_name = f.filename + f.filename.split(".")[-1]
    f.save(file_name)
    predict_and_extract(file_name)

    # Introduce a delay of 10 seconds
    time.sleep(10)

    # Redirect to download.html after uploading
    return redirect("/download.html")

# Route for downloading the file after delay
@app.route("/download")
def download():
    return send_file('/content/output.zip', as_attachment=True)

# Route to render download.html
@app.route("/download.html")
def download_html():
    return render_template("download.html")

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

app.run()

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
 * ngrok tunnel "https://8ef8-34-145-65-63.ngrok-free.app" -> "http://127.0.0.1:5000/"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
